# The Chinook media store database - Answering business questions using SQL

`category: business data analysis, expl. data analysis, intermediate SQL`

The Chinook database is a fictional database representing a digital media store. It is a well-known resource for practising with SQL, having a consistent number of tables, being well designed and maintained.<br/>
The database can be downloaded [here](https://www.sqlitetutorial.net/sqlite-sample-database/).

In this project, we're going to answer some business questions using SQLite, pretending to be data analysts in a digital music company.

The **project's goals** are to practice named subqueries, multiple joins and to write clear and well-organised queries.

## Getting to know the database schema

First of all, let's load the **ipython-sql** module and connect our notebook to the database.<br/> 
This way, we can write SQL queries directly in the cells by using the `%%sql` magic.

In [5]:
%%capture
%load_ext sql
%sql sqlite:///datasets/chinook.db

The tables and views included in the database are:

In [14]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('view', 'table');

 * sqlite:///datasets/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There are 11 tables and no views.<br/>
The tables contain information about employees, customers, the details of each invoice and about the tracks available in the store, including the albums and playlists to whom they belong.

The following image represents the **database schema**:

<img src="imgs/chinook-schema.svg" width=600 height=450 />

## 1) Selecting albums to purchase

Suppose our store have to choose **three** out of four new albums to purchase.<br/> 
We just know the name of the artists and the genres they belong to:

- **Name** / **Genre**
* Regal - Hip-Hop
* Red Tone - Punk
* Meteor and the Girls - Pop
* Slim Jim Bites - Blues

The record label from which our company will buy the albums wants us to advertise their artists in the USA market, so we have to decide by considering the relative popularity of the genres in the USA.

In [20]:
%%sql

WITH individual_purchases_usa_each_genre AS
    (
    SELECT 
        g.name genre, 
        il.quantity
    FROM customer c
        INNER JOIN invoice i ON c.customer_id = i.customer_id
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE c.country = 'USA'
    )
    
SELECT
    genre,
    SUM(quantity) no_of_tracks_sold,
    ROUND((SUM(quantity) / (
                          SELECT CAST(SUM(quantity) AS FLOAT)
                          FROM individual_purchases_usa_each_genre
                          )) * 100, 1) pct_sales
FROM individual_purchases_usa_each_genre
GROUP BY 1 
ORDER BY 2 DESC;

 * sqlite:///datasets/chinook.db
Done.


genre,no_of_tracks_sold,pct_sales
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


Based on the results of the query, we should buy **The Red Tone**, the **Slim Jim Bites** and the **Meteor and the Girls** albums. We also note that the sales percentage of Blues, Pop and Hip-Hop  are quite close to each other. Thus, other factors might influence our decision (ex. sales of previous records from the same artists).

## 2) Analyzing employees sales performance

Each customer is assigned to a **sales support agent** when he/she purchases a track for the first time.<br/>
Our company has three sales support agent. Their IDs are 3, 4 and 5:

In [21]:
%%sql

SELECT 
    employee_id, 
    (first_name || ' ' || last_name) name, 
    lower(title) title
FROM employee
WHERE title LIKE '%support%';

 * sqlite:///datasets/chinook.db
Done.


employee_id,name,title
3,Jane Peacock,sales support agent
4,Margaret Park,sales support agent
5,Steve Johnson,sales support agent


We want to analyse their sales performance, based on the total revenues of each.

In [25]:
%%sql

WITH sales_support_agent_performance AS
     (
     SELECT 
        e.employee_id, 
        (e.first_name || ' ' || e.last_name) name,
        DATE(e.hire_date) hire_date, 
        i.customer_id,  
        i.total invoice_total
     FROM employee e 
        INNER JOIN customer c ON e.employee_id = c.support_rep_id
        INNER JOIN invoice i ON c.customer_id = i.customer_id
     )
    
SELECT 
    employee_id sales_agent_id,
    name,
    hire_date,
    ROUND(SUM(invoice_total)) grand_total,
    COUNT(DISTINCT(customer_id)) no_of_cust_assigned,
    ROUND(SUM(invoice_total) / COUNT(DISTINCT(customer_id)), 0) avg_total_per_cust
FROM sales_support_agent_performance
GROUP BY 1
ORDER BY 4 DESC; 

 * sqlite:///datasets/chinook.db
Done.


sales_agent_id,name,hire_date,grand_total,no_of_cust_assigned,avg_total_per_cust
3,Jane Peacock,2017-04-01,1732.0,21,82.0
4,Margaret Park,2017-05-03,1584.0,20,79.0
5,Steve Johnson,2017-10-17,1394.0,18,77.0


If we look at the sheer sales revenues (i.e. the `grand_total` column), **Jane Peacock** is the top employee.<br/>
We included  the `hire_date` in the result set because it certainly can have an impact on the total of revenues. We see that Jane Peacock was hired before Margaret Park, who was in turn hired before Steve Johnson. As a consequence, **we cannot say that the performances of the sales support agents differ significantly**.

## 3) Sales by country

We have been asked to analyze the sales for each country in the database, by including number of customers, total sales, average revenue per customer and average order value.

In [26]:
%%sql

    /* selecting raw data by joining */
WITH country_all_sales AS
    (
     SELECT 
        c.country,
        c.customer_id,
        i.total
     FROM customer c
        INNER JOIN invoice i ON c.customer_id = i.customer_id
    ),
    
    /* categorization of only-one-customer countries as 'Other' */
    country_all_sales_other AS
    (
     SELECT 
        CASE 
            WHEN (
                  SELECT COUNT(customer_id)
                  FROM customer c
                  WHERE c.country = cas.country
                 ) = 1 THEN 'Other'
            ELSE country
        END AS country,
        cas.customer_id,
        cas.total
     FROM country_all_sales cas
    ),
    
    /* adding a flag = 1 for 'Other'*/
    country_all_sales_other_flag AS
    (
     SELECT 
        caso.*,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS flag        
     FROM country_all_sales_other caso   
    )    
    
    /* summary statistics */
SELECT 
    country,
    COUNT(DISTINCT(customer_id)) no_of_cust,
    ROUND(SUM(total), 0) total_sales,
    ROUND(SUM(total) / COUNT(DISTINCT(customer_id)), 0) avg_sale_per_customer,
    ROUND(AVG(total), 1) avg_order_value
FROM country_all_sales_other_flag 
GROUP BY country
ORDER BY flag, total_sales DESC;

 * sqlite:///datasets/chinook.db
Done.


country,no_of_cust,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.0,80.0,7.9
Canada,8,536.0,67.0,7.0
Brazil,5,428.0,86.0,7.0
France,5,389.0,78.0,7.8
Germany,4,335.0,84.0,8.2
Czech Republic,2,273.0,137.0,9.1
United Kingdom,3,246.0,82.0,8.8
Portugal,2,185.0,93.0,6.4
India,2,183.0,92.0,8.7
Other,15,1095.0,73.0,7.4


We see that the USA are our main market if we look at the `total_sales` column (units: $).<br/>
On the other hand, a good indicator of a potentially good market on which to invest in the future may be the **average value sales per customer**. Our table shows that:
* Czech Republic
* Portugal and India

are promising countries.

*(Of course, we note that our database contain data on a small scale. The conclusions based on such small samples are usually subject to enormous fluctuations and this has to be taken into consideration)*

We include <ins>a slight variation of the previous query</ins>, producing the same results. This is because these notebooks also serve as notes for future refreshers.

In [27]:
%%sql

WITH all_sales_country_other AS
    (
    SELECT 
        CASE
            WHEN (
                  SELECT COUNT(DISTINCT(customer_id))
                  FROM customer
                  WHERE country = c.country
                 ) = 1 THEN 'Other'
            ELSE c.country
            END AS country,
        c.customer_id,
        i.total    
    FROM customer c
        INNER JOIN invoice i ON c.customer_id = i.customer_id
    ),
   
    /* generate a new flag column to force 'Other' to the bottom in the final results */
    all_sales_country_other_flag AS
    (
    SELECT 
        asco.*,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS flag    
    FROM all_sales_country_other asco    
    )
    
    /* summary sales statistics for each country */
SELECT 
    country,
    COUNT(DISTINCT(customer_id)) no_of_cust,
    ROUND(SUM(total), 0) total_sales,
    ROUND(SUM(total) / COUNT(DISTINCT(customer_id)), 0) avg_sale_per_customer,
    ROUND(AVG(total), 1) avg_order_value
FROM all_sales_country_other_flag 
GROUP BY country
ORDER BY flag, total_sales DESC; 

 * sqlite:///datasets/chinook.db
Done.


country,no_of_cust,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.0,80.0,7.9
Canada,8,536.0,67.0,7.0
Brazil,5,428.0,86.0,7.0
France,5,389.0,78.0,7.8
Germany,4,335.0,84.0,8.2
Czech Republic,2,273.0,137.0,9.1
United Kingdom,3,246.0,82.0,8.8
Portugal,2,185.0,93.0,6.4
India,2,183.0,92.0,8.7
Other,15,1095.0,73.0,7.4


## 4) Albums vs single tracks: a change in strategy?

On our digital platform, customers can purchase:
* whole albums
* individual tracks

There is no overcharge for choosing one or the other option.
<br/> Our management is thinking about **purchasing only the most popular tracks from each album**, instead of the whole album. This potentially could lead to increased revenues if album purchases are relatively rare.

We'll write a query to find the relative frequency of the two purchase types.<br/>
The idea behind the query is to find the list of tracks of each invoice and then comparing it to the list of tracks in one of the albums linked to the purchase. The comparison will be done through a <ins>symmetric difference between the two sets</ins>.

In [29]:
%%sql

WITH album_track AS
    (
    SELECT 
        t.track_id,
        a.album_id
    FROM album a
        INNER JOIN track t ON a.album_id = t.album_id
    ),
    
    invoice_album_track AS
    (
    SELECT
        i.invoice_id,
        i.total,
        il.track_id,
        t.album_id
    FROM invoice i
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
    ),
    
    invoices_purchase_type AS
    (
    SELECT
        iat.*,
        CASE 
           WHEN (
                SELECT track_id
                FROM invoice_album_track
                WHERE invoice_id = iat.invoice_id

                EXCEPT 

                SELECT track_id
                FROM album_track
                WHERE album_id = iat.album_id 
                ) IS NULL

                AND 

                (
                SELECT track_id
                FROM album_track
                WHERE album_id = iat.album_id 

                EXCEPT

                SELECT track_id
                FROM invoice_album_track
                WHERE invoice_id = iat.invoice_id
                ) IS NULL then 'Album' 
            ELSE 'Tracks'     
            END AS purch_type       
    FROM invoice_album_track iat
    ),
    
    final_data AS
    (
    SELECT 
        invoice_id,
        total,
        purch_type
    FROM invoices_purchase_type    
    GROUP BY invoice_id
    )
    
SELECT
    purch_type,
    COUNT(*) no_invoices,
    ROUND((cast(count(*) as float) / (
                                      SELECT COUNT(*) 
                                      FROM final_data)
                                     ) * 100, 0) pct_invoices,
    ROUND(SUM(total), 0) tot_revenue,
    ROUND((SUM(total) / (
                         SELECT SUM(total) 
                         FROM final_data)
                        ) * 100, 0) pct_revenue
FROM final_data
GROUP BY purch_type;

 * sqlite:///datasets/chinook.db
Done.


purch_type,no_invoices,pct_invoices,tot_revenue,pct_revenue
Album,114,19.0,1461.0,31.0
Tracks,500,81.0,3248.0,69.0


The results show that both purchase options are chosen often.<br/>
Album purchases are responsible of more than 30% of revenues.

Based on these data, **we don't advise the new purchase strategy**.

## 5) Most popular artist in playlists

We want to find the artist who appear on the greatest number of playlists.

In [30]:
%%sql

SELECT 
    artist,
    COUNT(*) playlist_appearances
FROM 
    (
    SELECT 
        ar.name artist, 
        p.name playlist_name
    FROM artist ar
        INNER JOIN album al ON ar.artist_id = al.artist_id
        INNER JOIN track t ON al.album_id = t.album_id
        INNER JOIN playlist_track pt ON t.track_id = pt.track_id
        INNER JOIN playlist p ON pt.playlist_id = p.playlist_id
    GROUP BY 1,2
    ) artist_playlist 
GROUP BY 1
ORDER BY 2 DESC, 1
LIMIT 5;

 * sqlite:///datasets/chinook.db
Done.


artist,playlist_appearances
Eugene Ormandy,6
Academy of St. Martin in the Fields & Sir Neville Marriner,5
Berliner Philharmoniker & Herbert Von Karajan,5
English Concert & Trevor Pinnock,5
The King's Singers,5


The hungarian director and violinist Eugene Ormandy is the most popular artist in playlists.<br/>
The following are also classical music orchestras/groups.

We could advertise classical musician playlists on our mobile app (supposing there's one).

## 6) Purchased vs not purchased tracks

We want to compare the share of unpurchased tracks to the purchased ones.<br/>
We first **create a view** with each track id, its genre and a flag for purchased/unpurchased:

In [31]:
%%sql

/* DROP VIEW tracks_purchased_or_not */

CREATE VIEW tracks_purchased_or_not AS
    SELECT
        t.track_id,
        g.name,    
        CASE 
            WHEN t.track_id IN (
                                SELECT DISTINCT(track_id)
                                FROM invoice_line il
                               )
            THEN 'yes'
            ELSE 'no'
            END AS purchased
    FROM track t 
        INNER JOIN genre g ON t.genre_id = g.genre_id;

 * sqlite:///datasets/chinook.db
Done.


[]

We are ready to run our intial query.

In [32]:
%%sql

SELECT 
    purchased,
    COUNT(*) no_of_tracks,
    ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM track), 2) pct
FROM
    (
     tracks_purchased_or_not
    )
GROUP BY purchased;  

 * sqlite:///datasets/chinook.db
Done.


purchased,no_of_tracks,pct
no,1697,0.48
yes,1806,0.52


Because unpurchased tracks are 48% of our entire catalogue, we are interested to further investigate the problem. We want to calculate the share of unpurchased/purchased tracks for each genre.

In [33]:
%%sql

SELECT purch.genre,
       ROUND(not_purch.no_of_tracks / CAST(purch.no_of_tracks AS FLOAT), 2) 'not_purch/purch'
FROM    (
        SELECT
            name genre,
            COUNT(*) no_of_tracks
        FROM tracks_purchased_or_not
        WHERE purchased = 'yes'
        GROUP BY genre
        ) purch 
    
    INNER JOIN 
    
        (
        SELECT
            name genre,
            COUNT(*) no_of_tracks
        FROM tracks_purchased_or_not
        WHERE purchased = 'no'
        GROUP BY genre
        ) not_purch 
    
    ON purch.genre = not_purch.genre
ORDER BY 2 DESC;

 * sqlite:///datasets/chinook.db
Done.


genre,not_purch/purch
Drama,63.0
TV Shows,45.5
Soundtrack,7.6
Latin,3.87
Classical,3.63
Heavy Metal,3.0
Reggae,1.64
Jazz,1.13
Pop,0.92
Alternative & Punk,0.89


We see that genres from `Drama` to `Jazz` have a number of unpurchased tracks greater than the purchased.<br/>
On these genres, **it could be sensible to purchase only the most popular tracks from each album**, in order to minimize the ratio.

## 7) Protected vs non-protected media types sales

Let's look at the table `media_type` in order to understand the types of files we have in the catalogue:

In [34]:
%%sql

SELECT * FROM media_type;

 * sqlite:///datasets/chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


Media types 2, 3 and 4 are protected media types (you cannot share the corresponding files freely); 1 and 5 are non-protected. 

We want to assess whether there are differences in popularity between these two categories of media.<br/>
The popularity of each track is defined as the number of copies of its file which have been sold, considering all the invoices in the database. In other words, the column `quantity` in the `invoice_line` table will be used as the base measure of popularity.

In [35]:
%%sql

WITH popularity_media_type AS
    (
    SELECT 
        il.quantity,
        CASE
            WHEN mt.name LIKE 'P%' THEN 'Protected'
            ELSE 'Non-protected'
        END AS media_type_protection
    FROM invoice_line il
        INNER JOIN track t ON il.track_id = t.track_id
        INNER JOIN media_type mt ON t.media_type_id = mt.media_type_id
    )
    
SELECT 
    media_type_protection,
    SUM(quantity) popularity
FROM popularity_media_type    
GROUP BY 1;

 * sqlite:///datasets/chinook.db
Done.


media_type_protection,popularity
Non-protected,4280
Protected,477


Non-protected media types are **9 times** more popular than protected media types.

## Conclusions

In this project we practised some intermediate level SQL queries in order to answer some business questions about our digital media store activity.

We found results which can be the base of data-driven decisions for our company.

<ins>This project is part of the 'Working with data sources' course series on Dataquest [Data Scientist career path].</ins>